
# 📓 Tubería de Ingesta y Análisis Cualitativo de Entrevistas (DOCX → Embeddings → Qdrant/Neo4j/PostgreSQL)

**Fecha de generación:** 2025-10-29T22:13:14Z

Este cuaderno implementa una **tubería reproducible** para:

- **Extraer** texto desde entrevistas en `.docx` (párrafos/fragmentos).
- **Generar embeddings** con *Azure OpenAI* usando **deployments** (no modelos base).
- **Cargar y sincronizar** cada fragmento en tres almacenes complementarios:
  - **Qdrant (vector DB)** para búsqueda semántica/similaridad.
  - **Neo4j (grafo)** para modelar relaciones `(:Entrevista)-[:TIENE_FRAGMENTO]->(:Fragmento)`.
  - **PostgreSQL (relacional)** para consulta estructurada, auditorías y reporting (`embedding DOUBLE PRECISION[]`).

Además, incluye **celdas de apoyo para el análisis cualitativo** (Etapas 0–9) con prompts estandarizados para:
- Resumen/descripción inicial.
- Codificación abierta y axial (tablas/matrices).
- Codificación selectiva e integración narrativa.
- Análisis temático transversal y **modelo explicativo** (diagrama ASCII).
- Opciones para **persistir matrices** en PostgreSQL.

> ⚠️ **Seguridad**: No pegues tus claves directamente en el cuaderno. Usa un archivo `.env` local, variables de entorno o Azure Entra ID. Si previamente expusiste claves en notebooks, **rota** esas credenciales.


In [ ]:

# (Opcional) Instalar dependencias si hace falta (comenta si ya están instaladas)
# Nota: Ejecuta en tu entorno con internet.
# %pip install -U python-dotenv python-docx qdrant-client neo4j psycopg2-binary openai azure-identity azure-storage-blob tqdm tenacity



## 0. Configuración y credenciales (.env)

Crea un archivo `.env` en la carpeta del cuaderno con variables como:

```env
# Azure OpenAI
AZURE_OPENAI_ENDPOINT=https://<tu-recurso>.openai.azure.com
AZURE_OPENAI_API_KEY=xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx
AZURE_OPENAI_API_VERSION=2024-08-01-preview
AZURE_OPENAI_DEPLOYMENT_EMBED=text-embedding-3-large
AZURE_OPENAI_DEPLOYMENT_CHAT=gpt-4o-mini

# Qdrant
QDRANT_URI=https://<cluster-id>.<region>.cloud.qdrant.io
QDRANT_API_KEY=xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx

# Neo4j
NEO4J_URI=neo4j+s://<id>.databases.neo4j.io
NEO4J_USERNAME=neo4j
NEO4J_PASSWORD=xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx
NEO4J_DATABASE=neo4j

# PostgreSQL
PGHOST=localhost
PGPORT=5432
PGUSER=postgres
PGPASSWORD=<tu_password>
PGDATABASE=system_inv_sociocultural_v1

# Colección Qdrant y dimensiones
QDRANT_COLLECTION=entrevistas
EMBED_DIMS=3072  # text-embedding-3-large
```

> **Importante**: El endpoint de Azure OpenAI **debe** terminar en `.openai.azure.com`. Un endpoint como `...cognitive.microsoft.com` _no_ es válido para Azure OpenAI.


In [ ]:

# Crea un .env de ejemplo (no contiene secretos reales)
from pathlib import Path

env_example = Path("env.example")
if not env_example.exists():
    env_example.write_text(
        "AZURE_OPENAI_ENDPOINT=https://<tu-recurso>.openai.azure.com\n"
        "AZURE_OPENAI_API_KEY=REEMPLAZAR\n"
        "AZURE_OPENAI_API_VERSION=2024-08-01-preview\n"
        "AZURE_OPENAI_DEPLOYMENT_EMBED=text-embedding-3-large\n"
        "AZURE_OPENAI_DEPLOYMENT_CHAT=gpt-4o-mini\n"
        "QDRANT_URI=https://<cluster-id>.<region>.cloud.qdrant.io\n"
        "QDRANT_API_KEY=REEMPLAZAR\n"
        "NEO4J_URI=neo4j+s://<id>.databases.neo4j.io\n"
        "NEO4J_USERNAME=neo4j\n"
        "NEO4J_PASSWORD=REEMPLAZAR\n"
        "NEO4J_DATABASE=neo4j\n"
        "PGHOST=localhost\n"
        "PGPORT=5432\n"
        "PGUSER=postgres\n"
        "PGPASSWORD=REEMPLAZAR\n"
        "PGDATABASE=system_inv_sociocultural_v1\n"
        "QDRANT_COLLECTION=entrevistas\n"
        "EMBED_DIMS=3072\n"
    )
env_example.resolve()


## 1. Cargar configuración y preparar clientes

In [ ]:

import os, uuid, hashlib, time, json, math, re
from typing import List, Dict, Tuple
from dataclasses import dataclass
from dotenv import load_dotenv, find_dotenv

load_dotenv(find_dotenv(usecwd=True), override=True)

def mask(v: str, a: int = 4) -> str:
    if not v: return "****"
    return v[:a] + "…" + v[-a:] if len(v) > (2*a) else "****"

# --- Azure OpenAI ---
from openai import AzureOpenAI
from azure.identity import DefaultAzureCredential, get_bearer_token_provider

AZURE_OPENAI_ENDPOINT = os.getenv("AZURE_OPENAI_ENDPOINT") or ""
AZURE_OPENAI_API_KEY = os.getenv("AZURE_OPENAI_API_KEY") or ""
AZURE_OPENAI_API_VERSION = os.getenv("AZURE_OPENAI_API_VERSION", "2024-08-01-preview")
DEP_EMBED = os.getenv("AZURE_OPENAI_DEPLOYMENT_EMBED", "text-embedding-3-large")
DEP_CHAT  = os.getenv("AZURE_OPENAI_DEPLOYMENT_CHAT", "gpt-4o-mini")

if ".openai.azure.com" not in AZURE_OPENAI_ENDPOINT:
    print("⚠️  AVISO: AZURE_OPENAI_ENDPOINT no parece válido para Azure OpenAI:", AZURE_OPENAI_ENDPOINT)

if AZURE_OPENAI_API_KEY.strip():
    aoai = AzureOpenAI(azure_endpoint=AZURE_OPENAI_ENDPOINT, api_key=AZURE_OPENAI_API_KEY, api_version=AZURE_OPENAI_API_VERSION)
    print("Azure OpenAI (API key) listo →", AZURE_OPENAI_ENDPOINT)
else:
    scope = "https://cognitiveservices.azure.com/.default"
    credential = DefaultAzureCredential()
    token_provider = get_bearer_token_provider(credential, scope)
    aoai = AzureOpenAI(azure_endpoint=AZURE_OPENAI_ENDPOINT, azure_ad_token_provider=token_provider, api_version=AZURE_OPENAI_API_VERSION)
    print("Azure OpenAI (Entra ID) listo →", AZURE_OPENAI_ENDPOINT)

# --- Qdrant ---
from qdrant_client import QdrantClient
from qdrant_client.models import VectorParams, Distance, PointStruct

QDRANT_URI = os.getenv("QDRANT_URI")
QDRANT_API_KEY = os.getenv("QDRANT_API_KEY")
QDRANT_COLLECTION = os.getenv("QDRANT_COLLECTION", "entrevistas")

qdrant = QdrantClient(url=QDRANT_URI, api_key=QDRANT_API_KEY)
print("Qdrant:", QDRANT_URI)

# --- Neo4j ---
from neo4j import GraphDatabase
NEO4J_URI = os.getenv("NEO4J_URI"); NEO4J_USERNAME = os.getenv("NEO4J_USERNAME")
NEO4J_PASSWORD = os.getenv("NEO4J_PASSWORD"); NEO4J_DATABASE = os.getenv("NEO4J_DATABASE","neo4j")

neo = GraphDatabase.driver(NEO4J_URI, auth=(NEO4J_USERNAME, NEO4J_PASSWORD))
print("Neo4j:", NEO4J_URI)

# --- PostgreSQL ---
import psycopg2
from psycopg2.extras import execute_values

PGHOST=os.getenv("PGHOST","localhost"); PGPORT=int(os.getenv("PGPORT","5432"))
PGUSER=os.getenv("PGUSER","postgres"); PGPASSWORD=os.getenv("PGPASSWORD",""); PGDATABASE=os.getenv("PGDATABASE","postgres")

pg = psycopg2.connect(host=PGHOST, port=PGPORT, dbname=PGDATABASE, user=PGUSER, password=PGPASSWORD)
pg.set_client_encoding("UTF8")
pg_cur = pg.cursor()
print(f"PostgreSQL: {PGUSER}@{PGHOST}:{PGPORT}/{PGDATABASE}")

def get_embed_dim_fallback() -> int:
    # Intenta deducir dims del deployment de embeddings con una llamada mínima
    try:
        vec = aoai.embeddings.create(model=DEP_EMBED, input="ping").data[0].embedding
        return len(vec)
    except Exception as e:
        print("No se pudo inferir dimensión automáticamente:", e)
        return int(os.getenv("EMBED_DIMS", "3072"))  # fallback conocido para text-embedding-3-large

EMBED_DIMS = int(os.getenv("EMBED_DIMS", "0")) or get_embed_dim_fallback()
print("Dimensiones de embedding:", EMBED_DIMS)


## 2. Health checks y preparación de esquemas

In [ ]:

# Qdrant: asegurar colección con dimensiones correctas
def ensure_qdrant_collection(client: QdrantClient, name: str, dims: int, distance=Distance.COSINE) -> None:
    if client.collection_exists(name):
        info = client.get_collection(name)
        current = info.config.params.vectors.size
        if current != dims:
            raise RuntimeError(f"La colección '{name}' existe con size={current} y se requiere size={dims}. "
                               "Elimina/renombra y vuelve a crearla con el tamaño correcto.")
        return
    client.create_collection(name, vectors_config=VectorParams(size=dims, distance=distance))

ensure_qdrant_collection(qdrant, QDRANT_COLLECTION, EMBED_DIMS)
print(f"Qdrant OK → colección '{QDRANT_COLLECTION}' lista.")

# Neo4j: ping + constraints
with neo.session(database=NEO4J_DATABASE) as s:
    pong = s.run("RETURN 'pong' AS ping").single()["ping"]
    print("Neo4j ping:", pong)
    s.run("CREATE CONSTRAINT ent_nombre IF NOT EXISTS FOR (e:Entrevista) REQUIRE e.nombre IS UNIQUE")
    s.run("CREATE CONSTRAINT frag_id IF NOT EXISTS FOR (f:Fragmento) REQUIRE f.id IS UNIQUE")
    print("Constraints verificados.")

# PostgreSQL: crear tabla (embedding DOUBLE PRECISION[]) + índices
pg_cur.execute("""
CREATE TABLE IF NOT EXISTS entrevista_fragmentos (
  id TEXT PRIMARY KEY,
  archivo TEXT NOT NULL,
  par_idx INT NOT NULL,
  fragmento TEXT NOT NULL,
  embedding DOUBLE PRECISION[] NOT NULL,
  char_len INT NOT NULL,
  sha256 TEXT NOT NULL,
  created_at TIMESTAMPTZ NOT NULL DEFAULT NOW(),
  updated_at TIMESTAMPTZ NOT NULL DEFAULT NOW()
);
CREATE INDEX IF NOT EXISTS ix_ef_archivo ON entrevista_fragmentos(archivo);
CREATE INDEX IF NOT EXISTS ix_ef_charlen ON entrevista_fragmentos(char_len);
""")
pg.commit()
print("PostgreSQL OK → tabla 'entrevista_fragmentos' lista.")


## 3. Utilidades de lectura y segmentación (.docx → fragmentos)

In [ ]:

from docx import Document

def normalize_text(s: str) -> str:
    s = s.replace('\u00A0', ' ')  # NBSP → espacio
    s = re.sub(r'[ \t]+', ' ', s)
    s = re.sub(r'\s+\n', '\n', s)
    s = s.strip()
    return s

def read_paragraphs_from_docx(path: str) -> List[str]:
    try:
        d = Document(path)
        paras = [normalize_text(p.text) for p in d.paragraphs if p.text and p.text.strip()]
        return [p for p in paras if p]
    except Exception as e:
        print(f"❌ Error leyendo '{path}':", e)
        return []

def coalesce_small(paragraphs: List[str], min_chars: int = 200, max_chars: int = 1200) -> List[str]:
    """
    Une párrafos demasiado cortos para evitar fragmentos pobres, sin pasar el máximo.
    Heurística simple por longitud (chars).
    """
    acc, buf = [], ""
    for p in paragraphs:
        if not buf:
            buf = p
            continue
        if len(buf) < min_chars and len(buf) + 1 + len(p) <= max_chars:
            buf = buf + " " + p
        else:
            acc.append(buf)
            buf = p
    if buf:
        acc.append(buf)
    return acc

def make_fragment_id(file_name: str, idx: int) -> str:
    return str(uuid.uuid5(uuid.NAMESPACE_URL, f"{file_name}|{idx}"))

def batched(seq, size: int):
    buf = []
    for x in seq:
        buf.append(x)
        if len(buf) >= size:
            yield buf
            buf = []
    if buf:
        yield buf


## 4. Embeddings en batch (Azure OpenAI Deployments)

In [ ]:

def embed_batch(texts: List[str]) -> List[List[float]]:
    # Mantiene orden de entrada
    resp = aoai.embeddings.create(model=DEP_EMBED, input=texts)
    data_sorted = sorted(resp.data, key=lambda d: d.index)
    vecs = [d.embedding for d in data_sorted]
    # Sanidad rápida
    if any(len(v) != EMBED_DIMS for v in vecs):
        raise RuntimeError("Dimensiones de los embeddings no coinciden con EMBED_DIMS.")
    return vecs


## 5. Upsert en Qdrant / MERGE en Neo4j / INSERT en PostgreSQL

In [ ]:

def qdrant_upsert_points(client: QdrantClient, collection: str, ids: List[str],
                         files_and_frags: List[Tuple[str, int, str]],
                         vectors: List[List[float]]) -> None:
    pts = []
    for _id, (archivo, par_idx, frag), vec in zip(ids, files_and_frags, vectors):
        pts.append(PointStruct(
            id=str(_id),
            vector=vec,
            payload={
                "archivo": archivo,
                "par_idx": par_idx,
                "fragmento": frag,
                "char_len": len(frag)
            }
        ))
    client.upsert(collection_name=collection, points=pts)

def neo4j_merge(driver, database: str, rows: List[Dict[str, str]]) -> None:
    cypher = """
    UNWIND $rows AS r
    MERGE (e:Entrevista {nombre: r.archivo})
    MERGE (f:Fragmento {id: r.id})
      ON CREATE SET f.texto = r.fragmento, f.par_idx = r.par_idx, f.char_len = r.char_len
      ON MATCH  SET f.texto = coalesce(r.fragmento, f.texto), f.char_len = r.char_len
    MERGE (e)-[:TIENE_FRAGMENTO]->(f)
    """
    with driver.session(database=database) as s:
        s.run(cypher, rows=rows)

def pg_insert(pg_cur, rows: List[Tuple[str, str, int, str, list, int, str]]):
    sql = """
    INSERT INTO entrevista_fragmentos (id, archivo, par_idx, fragmento, embedding, char_len, sha256)
    VALUES %s
    ON CONFLICT (id) DO UPDATE SET
      fragmento = EXCLUDED.fragmento,
      embedding = EXCLUDED.embedding,
      char_len  = EXCLUDED.char_len,
      sha256    = EXCLUDED.sha256,
      updated_at = NOW();
    """
    execute_values(pg_cur, sql, rows, page_size=200)

def sha256_text(s: str) -> str:
    return hashlib.sha256(s.encode('utf-8')).hexdigest()


## 6. Orquestador de ingesta (DOCX → fragmentos → embeddings → 3 almacenes)

In [ ]:

from tqdm import tqdm

def ingest_files(file_paths: List[str], batch_size: int = 64, coalesce_min_chars: int = 200, coalesce_max_chars: int = 1200):
    ensure_qdrant_collection(qdrant, QDRANT_COLLECTION, EMBED_DIMS)
    total = 0
    for path in file_paths:
        archivo = os.path.basename(path)
        paras = read_paragraphs_from_docx(path)
        if not paras:
            print(f"⚠️  {archivo}: sin contenido utilizable.")
            continue

        frags = coalesce_small(paras, min_chars=coalesce_min_chars, max_chars=coalesce_max_chars)
        ids = [make_fragment_id(archivo, i) for i in range(len(frags))]
        file_triplets = [(archivo, i, frag) for i, frag in enumerate(frags)]
        print(f"📄 {archivo} → {len(frags)} fragmentos (fuente: {len(paras)} párrafos)")

        # embeddings en batch
        vectors = []
        for chunk in tqdm(list(batched(frags, batch_size)), desc=f"Embeddings {archivo}"):
            vectors.extend(embed_batch(chunk))

        # Qdrant
        qdrant_upsert_points(qdrant, QDRANT_COLLECTION, ids, file_triplets, vectors)

        # Neo4j
        neo_rows = [{
            "id": _id,
            "archivo": archivo,
            "fragmento": frag,
            "par_idx": idx,
            "char_len": len(frag)
        } for _id, (archivo, idx, frag) in zip(ids, file_triplets)]
        neo4j_merge(neo, NEO4J_DATABASE, neo_rows)

        # Postgres
        pg_rows = [(_id, archivo, idx, frag, vec, len(frag), sha256_text(frag))
                   for _id, (archivo, idx, frag), vec in zip(ids, file_triplets, vectors)]
        pg_insert(pg_cur, pg_rows); pg.commit()

        total += len(frags)
        print(f"✔️  {archivo}: {len(frags)} fragmentos cargados.")

    print(f"\n✅ Ingesta finalizada. Total fragmentos: {total}")


## 7. Lista de entrevistas (.docx)

In [ ]:

# Reemplaza con tus rutas locales en Windows/Linux/Mac
interview_files = [
    r"C:\ruta\a\tu\Entrevistas\Entre1.docx",
    r"C:\ruta\a\tu\Entrevistas\Entre2.docx",
    # ...
]

# (Opcional) Utilidad que sugiere 'Transcripción' si hay error tipográfico común
def exists_or_fix(path: str):
    import os
    if os.path.exists(path): return ("OK", path)
    alt = path.replace("Trancripción", "Transcripción")
    if alt != path and os.path.exists(alt): return ("SUGERIDO", alt)
    return ("NO_ENCONTRADO", path)

files_ok = []
for p in interview_files:
    status, use = exists_or_fix(p)
    print(f"{status:12} — {use}")
    if status != "NO_ENCONTRADO": files_ok.append(use)

print(f"➡️  Se procesarán {len(files_ok)} archivos.")


## 8. Ejecutar ingesta

In [ ]:

# Ejecuta la ingesta (descomenta para correr)
# ingest_files(files_ok, batch_size=64, coalesce_min_chars=200, coalesce_max_chars=1200)


## 9. Pruebas de consulta (Qdrant / Neo4j / Postgres)

In [ ]:

def semantic_search(query: str, top_k: int = 5):
    qvec = aoai.embeddings.create(model=DEP_EMBED, input=query).data[0].embedding
    res = qdrant.query_points(collection_name=QDRANT_COLLECTION, query=qvec, limit=top_k, with_payload=True)
    print(f"Consulta: {query}\nTop-{top_k} resultados:\n")
    for p in res.points:
        pl = p.payload or {}
        print(f"• score={p.score:.4f} | archivo={pl.get('archivo')} | par_idx={pl.get('par_idx')} | len={pl.get('char_len')}\n  “{(pl.get('fragmento') or '')[:180]}...”\n")

# Neo4j: conteo por entrevista
def graph_counts():
    with neo.session(database=NEO4J_DATABASE) as s:
        data = s.run("""
            MATCH (e:Entrevista)-[:TIENE_FRAGMENTO]->(f:Fragmento)
            RETURN e.nombre AS entrevista, count(f) AS n
            ORDER BY n DESC
        """).data()
    for row in data:
        print(f"{row['entrevista']}: {row['n']} fragmentos")


# Postgres: muestra 3 filas
def sample_pg(n: int = 3):
    pg_cur.execute("SELECT id, archivo, par_idx, char_len FROM entrevista_fragmentos LIMIT %s", (n,))
    for r in pg_cur.fetchall():
        print(r)

# Ejemplos (descomenta tras ingesta):
# semantic_search("conflictos de drenaje y crecimiento urbano", top_k=5)
# graph_counts()
# sample_pg(5)


## 10. Análisis cualitativo asistido (Etapas 0–9)

In [ ]:

QUAL_SYSTEM_PROMPT = """Eres un asistente AI experto en metodología cualitativa y análisis de entrevistas.
Se te suministrarán entrevistas transcritas previamente revisadas y verificadas en su fidelidad.
Tu tarea es guiar el análisis cualitativo siguiendo un enfoque sistemático en varias etapas, desde la preparación hasta la estructuración del informe final.

Instrucciones:
Etapa 0 – Preparación, Reflexividad y Configuración del Análisis: Revisa el texto buscando incoherencias en la transcripción.
Etapa 1 – Transcripción y resumen: Verifica literalidad y elabora un resumen breve.
Etapa 2 – Análisis Descriptivo Inicial: Resume primeras impresiones y temas superficiales. Justifica códigos iniciales.
Etapa 3 – Codificación Abierta: Propón códigos y citas (matriz).
Etapa 4 – Codificación Axial: Agrupa en categorías axiales, con notas/memos y relaciones.
Etapa 5 – Codificación Selectiva: Identifica el núcleo temático integrador.
Etapa 6 – Análisis Temático Transversal: Compara categorías por entrevista, señalando convergencias/divergencias y variaciones por rol/género/tiempo.
Etapa 7 – Modelo Explicativo: Propón un diagrama ASCII (mapa conceptual) con relaciones entre problemas urbanísticos, participación y transformación histórica.
Etapa 8 – Verificación/Validación/Saturación: Evalúa saturación, triangulación y factibilidad de member checking.
Etapa 9 – Hacia el Informe Final: Esboza estructura de informe con referencias a matrices, citas variadas (anonimizadas), limitaciones y recomendaciones.

Devuelve SIEMPRE un JSON *válido* con esta forma mínima:
{
  "etapa0_observaciones": "...",
  "etapa1_resumen": "...",
  "etapa2_descriptivo": { "impresiones": "...", "lista_codigos_iniciales": ["..."] },
  "etapa3_matriz_abierta": [ { "codigo": "...", "cita": "...", "fuente": "Entrevistado/a" } ],
  "etapa4_axial": [ { "categoria": "...", "codigos": ["..."], "relaciones": ["A->B", "B<->C"], "memo": "..." } ],
  "etapa5_selectiva": { "nucleo": "...", "narrativa": "..." },
  "etapa6_transversal": { "convergencias": "...", "divergencias": "...", "variaciones": "..." },
  "etapa7_modelo_ascii": "Texto de diagrama",
  "etapa8_validacion": { "saturacion": "...", "triangulacion": ["..."], "member_checking": "..." },
  "etapa9_borrador_informe": "Esquema de informe"
}
Usa citas literales cortas (≤ 40–60 palabras). Mantén anonimización.
"""


In [ ]:

def call_llm_chat_json(system_prompt: str, user_prompt: str, temperature: float = 0.2, max_tokens: int = 1800) -> dict:
    msg = [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_prompt}
    ]
    try:
        comp = aoai.chat.completions.create(
            model=DEP_CHAT,
            messages=msg,
            temperature=temperature
        )
        txt = comp.choices[0].message.content
        # Intentar extraer JSON – se tolera texto extra
        start = txt.find("{"); end = txt.rfind("}")
        if start != -1 and end != -1 and end > start:
            txt = txt[start:end+1]
        return json.loads(txt)
    except Exception as e:
        print("❌ LLM error:", e)
        return {}

def analyze_interview_text(text: str, fuente: str) -> dict:
    # Acota longitud si es necesario; para contextos largos se puede trocear y resumir primero
    user_prompt = f"""Analiza la siguiente transcripción (fuente: {fuente}) siguiendo las Etapas 0–9. 
Devuelve SOLO el JSON. 
Texto:
"""{text}"""
"""
    return call_llm_chat_json(QUAL_SYSTEM_PROMPT, user_prompt)


In [ ]:

import pandas as pd

def matriz_etapa3(json_out: dict) -> pd.DataFrame:
    rows = json_out.get("etapa3_matriz_abierta", []) or []
    df = pd.DataFrame(rows, columns=["codigo", "cita", "fuente"])
    df.rename(columns={"codigo":"Código Abierto", "cita":"Cita Textual / Ejemplo Relevante", "fuente":"Fuente (Entrevistado/a)"}, inplace=True)
    return df

def matriz_etapa4(json_out: dict) -> pd.DataFrame:
    rows = json_out.get("etapa4_axial", []) or []
    # Expandir filas por código para facilitar tabla relacional
    exp = []
    for r in rows:
        categoria = r.get("categoria","")
        memo = r.get("memo","")
        relaciones = "; ".join(r.get("relaciones", []) or [])
        for c in r.get("codigos", []) or []:
            exp.append({"Categoría Axial": categoria, "Código Abierto": c, "Relaciones": relaciones, "Notas/Memos": memo})
    df = pd.DataFrame(exp, columns=["Categoría Axial", "Código Abierto", "Relaciones", "Notas/Memos"])
    return df


In [ ]:

def print_modelo_ascii(json_out: dict):
    print(json_out.get("etapa7_modelo_ascii", "(sin diagrama)"))


### Ejemplo: correr análisis sobre una entrevista (después de la ingesta)

In [ ]:

# Carga una entrevista para análisis (reemplaza la ruta)
# path = r"C:\ruta\a\tu\Entrevistas\Entre1.docx"
# text = "\n".join(read_paragraphs_from_docx(path))
# out = analyze_interview_text(text, fuente=os.path.basename(path))

# # Visualizar matrices
# df3 = matriz_etapa3(out); df4 = matriz_etapa4(out)
# display(df3.head(10))
# display(df4.head(10))
# print_modelo_ascii(out)


## 11. (Opcional) Persistir matrices de análisis en PostgreSQL

In [ ]:

def ensure_analysis_tables():
    pg_cur.execute("""
    CREATE TABLE IF NOT EXISTS analisis_codigos_abiertos (
        archivo TEXT NOT NULL,
        codigo TEXT NOT NULL,
        cita TEXT NOT NULL,
        fuente TEXT NOT NULL,
        created_at TIMESTAMPTZ NOT NULL DEFAULT NOW()
    );
    CREATE TABLE IF NOT EXISTS analisis_axial (
        archivo TEXT NOT NULL,
        categoria TEXT NOT NULL,
        codigo TEXT NOT NULL,
        relaciones TEXT,
        memo TEXT,
        created_at TIMESTAMPTZ NOT NULL DEFAULT NOW()
    );
    """)
    pg.commit()

def persist_analysis(archivo: str, json_out: dict):
    ensure_analysis_tables()
    # Etapa 3
    rows3 = json_out.get("etapa3_matriz_abierta", []) or []
    if rows3:
        execute_values(pg_cur,
            "INSERT INTO analisis_codigos_abiertos (archivo, codigo, cita, fuente) VALUES %s",
            [(archivo, r.get("codigo",""), r.get("cita",""), r.get("fuente","")) for r in rows3]
        )
    # Etapa 4
    rows4 = json_out.get("etapa4_axial", []) or []
    exp4 = []
    for r in rows4:
        categoria = r.get("categoria",""); memo = r.get("memo","")
        relaciones = "; ".join(r.get("relaciones", []) or [])
        for c in r.get("codigos", []) or []:
            exp4.append((archivo, categoria, c, relaciones, memo))
    if exp4:
        execute_values(pg_cur,
            "INSERT INTO analisis_axial (archivo, categoria, codigo, relaciones, memo) VALUES %s",
            exp4
        )
    pg.commit()


## 12. Cierre de conexiones (cuando termines)

In [ ]:

try:
    pg_cur.close(); pg.close()
except Exception:
    pass
try:
    neo.close()
except Exception:
    pass
print("🔚 Conexiones cerradas.")
